In [2]:
%pylab inline

import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


# Setup the data

In [3]:
seed = 128
rng = np.random.RandomState(seed)

In [4]:
root_dir = os.path.abspath('..')
data_dir = os.path.join(root_dir, 'data')

# check for existence
os.path.exists(root_dir)
os.path.exists(data_dir)

True

In [5]:
train = pd.read_csv(os.path.join(data_dir, 'train.csv'), dtype=np.float32)
test = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None, dtype=np.float32)

In [6]:
train_x = train.iloc[:, :-1].values
test_x = test.iloc[:, :].values

In [7]:
split_size = int(train_x.shape[0]*0.7)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train.popularity.values[:split_size], train.popularity.values[split_size:]

# Auxilary functions

In [8]:
def dense_to_one_hot(labels_dense, num_classes=4):
    """Convert class labels from scalars to one-hot vectors"""
    s = pd.Series(list(range(1,num_classes+1,1)))
    labels_one_hot = pd.get_dummies(labels_dense)
    
#     num_labels = labels_dense.shape[0]
#     index_offset = np.arange(num_labels) * num_classes
#     labels_one_hot = np.zeros((num_labels, num_classes))
#     labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

def preproc(unclean_batch_x):
    """Convert values to range 0-1, with mean centered"""
    temp_batch = (unclean_batch_x-unclean_batch_x.min()) / (unclean_batch_x.max()-unclean_batch_x.min())
    
    return temp_batch

def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    
    batch_x = eval(dataset_name + '_x')[[batch_mask]].reshape(-1, input_num_units)
    batch_x = preproc(batch_x)
    
    if dataset_name == 'train':
        batch_y = eval(dataset_name).ix[batch_mask, 'popularity'].values
        batch_y = dense_to_one_hot(batch_y)
        
    return batch_x, batch_y

# Neural Net definition

In [37]:
### set all variables

# number of neurons in each layer
input_num_units = 6
hidden_num_units_1 = 100
hidden_num_units_2 = 100
output_num_units = 4

# define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])

# set remaining variables
epochs = 250
batch_size = 512
learning_rate = 0.01

### define weights and biases of the neural network (refer this article if you don't understand the terminologies)

weights = {
    'hidden_1': tf.Variable(tf.random_normal([input_num_units, hidden_num_units_1], seed=seed)),
    'hidden_2': tf.Variable(tf.random_normal([hidden_num_units_1, hidden_num_units_2], seed=seed)),
    'output': tf.Variable(tf.random_normal([hidden_num_units_2, output_num_units], seed=seed))
}

biases = {
    'hidden_1': tf.Variable(tf.random_normal([hidden_num_units_1], seed=seed)),
    'hidden_2': tf.Variable(tf.random_normal([hidden_num_units_2], seed=seed)),
    'output': tf.Variable(tf.random_normal([output_num_units], seed=seed))
}

### Computational Graph

In [38]:
hidden_layer_1 = tf.add(tf.matmul(x, weights['hidden_1']), biases['hidden_1'])
hidden_layer_1 = tf.nn.relu(hidden_layer_1)
hidden_layer_2 = tf.add(tf.matmul(hidden_layer_1, weights['hidden_2']), biases['hidden_2'])
hidden_layer_2 = tf.nn.relu(hidden_layer_2)

output_layer = tf.matmul(hidden_layer_2, weights['output']) + biases['output']


### Cost function and Optimizer

In [39]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_layer))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [40]:
init = tf.global_variables_initializer()

In [41]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        avg_cost = 0
        total_batch = int(train.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
            _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
            
            avg_cost += c / total_batch
            
        print( "Epoch:", (epoch+1), "cost =", "{:.5f}".format(avg_cost) )
    
    print( "\nTraining complete!" )
    
    
    # find predictions on val set
    pred_temp = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
    print( "Validation Accuracy:", accuracy.eval({x: val_x.reshape(-1, input_num_units), y: dense_to_one_hot(val_y)}) )
    
    predict = tf.argmax(output_layer, 1)
    pred = predict.eval({x: test_x.reshape(-1, input_num_units)})

Epoch: 1 cost = 15.98574
Epoch: 2 cost = 9.91691
Epoch: 3 cost = 7.93598
Epoch: 4 cost = 5.46604
Epoch: 5 cost = 4.48811
Epoch: 6 cost = 3.53503
Epoch: 7 cost = 2.85252
Epoch: 8 cost = 2.12825
Epoch: 9 cost = 2.40547
Epoch: 10 cost = 2.33663
Epoch: 11 cost = 1.50669
Epoch: 12 cost = 1.12900
Epoch: 13 cost = 1.16287
Epoch: 14 cost = 0.88149
Epoch: 15 cost = 0.86997
Epoch: 16 cost = 0.75072
Epoch: 17 cost = 0.61069
Epoch: 18 cost = 0.51549
Epoch: 19 cost = 0.43406
Epoch: 20 cost = 0.43250
Epoch: 21 cost = 0.34173
Epoch: 22 cost = 0.35016
Epoch: 23 cost = 0.35800
Epoch: 24 cost = 0.35546
Epoch: 25 cost = 0.29966
Epoch: 26 cost = 0.28586
Epoch: 27 cost = 0.27902
Epoch: 28 cost = 0.25076
Epoch: 29 cost = 0.23185
Epoch: 30 cost = 0.20206
Epoch: 31 cost = 0.21123
Epoch: 32 cost = 0.17013
Epoch: 33 cost = 0.17481
Epoch: 34 cost = 0.19364
Epoch: 35 cost = 0.21177
Epoch: 36 cost = 0.20383
Epoch: 37 cost = 0.16818
Epoch: 38 cost = 0.17998
Epoch: 39 cost = 0.13636
Epoch: 40 cost = 0.17146
Epoch: 4

In [42]:
np.savetxt("out.txt", pred, fmt="%d",)